In [ ]:
!pip install  -U -q transformers trl datasets bitsandbytes peft accelerate
# Tested with transformers==4.46.3, trl==0.12.2, datasets==3.2.0, bitsandbytes==0.45.0, peft==0.14.0, accelerate==1.2.0

In [ ]:
!pip install -q flash-attn --no-build-isolation

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

**Prepare Your Custom Dataset**

In [ ]:
from PIL import Image
import os
import io
import matplotlib.pyplot as plt

# Create a list to store all the image data
train_dataset = []

# Paths to your 100 images
image_directory = "/content/drive/MyDrive/SMOL_VLM_Images"  # Adjust the path to your image folder
image_files = os.listdir(image_directory)

chosen_responses = [
    "Yes, the wood in the image appears to be caught in fire. This could indicate that the photo was taken during a dry season or in a region that experiences arid conditions. The brown burnt woods contrasts with the color of the coal and provides a orange flame that highlights that the fire is being burnt for soaking heat.",
    "Yes, the fire in the chimney seems to be burning brightly. This could be indicative of a heating system working effectively, or possibly due to intense wind conditions which can cause flames to spread quickly, making the fire appear more dramatic in the image.",
    "Yes, the image shows trees with visible flames around them. The fire could have been caused by natural factors like lightning strikes, or it might have been ignited by human activity. The trees seem to be burning intensely, likely due to dry conditions in the area.",
    "Yes, the building appears to have a fire on the interior, which is visible through the windows. This could be the result of an electrical issue, a kitchen fire, or some other source of combustion within the building. It’s important for the safety response teams to be alerted to contain it.",
    "Yes, the farm house in the image is likely on fire. This could be due to a variety of reasons such as equipment malfunction, lightning strikes, or an unattended open flame. The image shows smoke rising from the roof, indicating the presence of active flames inside.",
    "Yes, the nursery appears to be on fire. This might have been caused by an accident, such as a malfunctioning heating system or a nearby wildfire that has spread to the structure. The flames are visible around the roof and windows.",
    "Yes, the image shows a large fire. It seems like a significant fire event, possibly in an urban or industrial area. The fire appears to be spreading rapidly, and there is a thick layer of smoke indicating the severity of the situation.",
    "Yes, the building is on fire. The flames are visible at multiple points of the building, suggesting a fast-spreading fire. The building might have caught fire due to an electrical malfunction, heating equipment, or a human error.",
    "Yes, the fire is visible behind the building, likely from a fire source such as a nearby forest or an external structure. The image shows thick black smoke rising, which is often associated with a large-scale fire.",
    "Yes, there is a small fire with faint smoke in the image. The fire doesn’t seem to be intense, but the presence of smoke could indicate that it’s in the early stages. This type of fire may have started from discarded materials or a small appliance.",
    "Yes, parts of the image show fire caught in small areas. The fire seems to be spreading, but the intensity is localized, with smaller areas of combustion scattered throughout the scene.",
    "Yes, the building appears to be on fire, with flames visible on the exterior. This could be due to a variety of causes, such as a gas leak, electrical failure, or arson.",
    "Yes, the factory's large chimneys are engulfed in a massive fire. This could have been caused by an industrial accident or a malfunction in one of the large equipment used at the factory, creating significant fire hazards.",
    "Yes, there appears to be fire behind the houses in the image. It could be a wildfire or an uncontrolled fire in the area that is spreading toward the homes, which could pose a risk to residents.",
    "Yes, there is fire coming out from behind the building. This might be the result of an accident or a secondary fire source, such as a gas tank explosion or an electrical short.",
    "Yes, the building is on fire, and the flames seem to be spreading throughout the structure. This could have been caused by a kitchen fire, an electrical fault, or external factors like lightning or arson.",
    "Yes, there is a fire in the construction area. This could be caused by construction equipment malfunction, sparks from welding, or the presence of flammable materials on-site. The fire seems to have ignited quickly.",
    "Yes, the house appears to be on fire. This could be a result of electrical failure, gas leak, or negligence, and it’s important that firefighters respond quickly to prevent the fire from spreading.",
    "Yes, the roof of the house appears to be on fire. This could indicate that the fire has been smoldering for some time or that an external fire source has caused the roof to catch fire. The flames are clearly visible and seem to be climbing the sides.",
    "Yes, the roof of the building is on fire, and the intensity seems high. This could indicate a significant fire hazard inside the building. The image shows large flames engulfing the top portion of the structure.",
    "Yes, the inside back portion of the factory is on fire. It could be due to malfunctioning equipment, hazardous materials, or poor maintenance. The flames are visible from the exterior of the factory.",
    "Yes, the fire seems to be quite intense, with large flames visible across the image. The fire might have spread from one source or may be multiple fires merging together, creating a massive blaze.",
    "Yes, there is fire on the roof of a house. The flames seem to be affecting the roofing materials, and this could pose a significant risk to the structure’s stability if not addressed immediately.",
    "Yes, there seems to be a fire in the city buildings, likely due to either an accident or intentional act. The fire appears to be spreading quickly, and firefighters should be on high alert.",
    "Yes, the house roof is on fire, and there is also smoke emerging from the flames. This could suggest that the fire is intense and potentially has been burning for some time before becoming visible.",
    "Yes, there seems to be a fire rapidly spreading across the area. The flames seem to be gaining intensity, possibly due to strong winds or an uncontrollable source of combustion."
]

rejected_responses = [
    "No, there doesn't seem to be any wood caught on fire in the image. The area shown in the photo appears to be safe, and no signs of combustion are visible.",
    "No, the fire doesn't seem to be burning in the chimney. The smoke in the image could be from a different source, and the chimney appears to be clear of any visible flames.",
    "No, the image does not show any trees caught in a fire. The trees appear to be intact, and there is no visible flame near them.",
    "No, there doesn't seem to be any fire inside the building. The interior looks undisturbed, and no flames or smoke are visible from the windows.",
    "No, the farm house in the image is not on fire. The smoke could be from a nearby area, but the building itself looks intact and unaffected by fire.",
    "No, the nursery is not on fire. The structure looks undamaged, and there are no visible flames or smoke around it.",
    "No, there is no large fire in the image. The scene looks calm, and the image does not depict any dangerous flames or large fires.",
    "No, the building in the image is not on fire. It appears to be in good condition with no visible flames or smoke around it.",
    "No, there is no fire behind the building. The area looks clear of any visible smoke or flames.",
    "No, there is no noticeable fire with smoke in the image. The environment seems to be free of any active flames.",
    "No, the fire in the image appears to be minimal. There are no significant visible flames or fire spreading across the scene.",
    "No, the building does not seem to be on fire. It looks intact with no visible flames or smoke.",
    "No, there is no visible fire in the factory chimneys. The chimneys appear clean and free from flames or hazardous conditions.",
    "No, there seems to be no fire behind the houses in the image. The area appears to be safe with no visible smoke or flames.",
    "No, there is no fire coming out behind the building. The area looks undisturbed, and there are no flames or smoke.",
    "No, there is no visible fire inside the building. The interior looks normal, and there is no indication of combustion.",
    "No, there is no fire in the construction area. The scene is clear, with no visible flames or smoke.",
    "No, the house is not on fire. The structure appears safe, and there are no signs of smoke or flames.",
    "No, the roof of the house is not on fire. There are no visible flames or damage to the structure.",
    "No, the building roof is not on fire. The flames seem to be in another part of the building or in a different structure altogether.",
    "No, there is no fire in the inside back portion of the factory. The factory looks intact, with no visible fire or smoke.",
    "No, the fire in the image does not appear to be intense. There are no large flames, and the fire appears to be under control.",
    "No, there is no fire on the roof of the house. The structure appears to be undamaged and safe from fire.",
    "No, there is no visible fire in the city buildings. The scene is clear and appears safe.",
    "No, the fire doesn’t seem to be spreading rapidly across the area. The scene appears calm, with no signs of a fast-moving blaze."
]

prompts = [
    "Does the wood caught with fire?",
    "How the fire caught in the chymney?",
    "Is there fire caught on the woods trees?",
    "Is the building caught with fire from inside?",
    "Does a farm house caught with fire?",
    "Is nursery caught with fire?",
    "Is there any fire in the image that is huge?",
    "Is the building caught with fire?",
    "Is there any fire behind the building?",
    "Is there any fire with a slight smoke around in the image?",
    "Is there any fire caught in imahe and caught in parts?",
    "Does the building caught with fire?",
    "Is there any big fire caught to the factrory big chymneys?",
    "Does there any fire behind the houses?",
    "Is there any fire coming out behind the building in the image?",
    "Is the building caught with fire?",
    "Is the fire caught in the construction area?",
    "Does there any fore caught in the house?",
    "Is there any building caught with fire bhind the building?",
    "Does the house roof caught with fire?",
    "Is the roof of the building caught with high fire?",
    "Is there any fire in the inside back portion of the factory?",
    "Is there any intense fire?",
    "Is there any fire on the roof of a house?",
    "Is tehre any fire in the city buildings?",
    "Does the house roof caught with fire and there is smoke too emerging from the fire?"
]

# Sort image files to ensure sequential processing
image_files = sorted(image_files)

# Loop through the image files and create a structured dataset
for i, image_file in enumerate(image_files):
    if image_file.lower().endswith(".jpg"):  # Case-insensitive check for .jpg extension
        # Open and convert the image to RGB
        img_path = os.path.join(image_directory, image_file)
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')  # Convert to RGB if not already in RGB mode

        # Resize the image to 500x333
        image = image.resize((500, 333))

        # Save image in JPEG format to buffer and re-open
        img_byte_array = io.BytesIO()
        image.save(img_byte_array, format='JPEG')
        img_byte_array.seek(0)  # Rewind the buffer to the start
        image_jpeg = Image.open(img_byte_array)

        # Validate indices for chosen_responses, rejected_responses, and prompts
        chosen_response = chosen_responses[i] if i < len(chosen_responses) else None
        rejected_response = rejected_responses[i] if i < len(rejected_responses) else None
        prompt_text = prompts[i] if i < len(prompts) else None

        # Structure for each image entry
        image_data = {
            "chosen": [{
                "content": [{
                    "text": chosen_response,
                    "type": "text"
                }],
                "role": "assistant"
            }],
            "rejected": [{
                "content": [{
                    "text": rejected_response,
                    "type": "text"
                }],
                "role": "assistant"
            }],
            "images": [image_jpeg],  # Store the image as a list with the correct format
            "prompt": [{
                "content": [{
                    "text": None,
                    "type": "image"
                }, {
                    "text": prompt_text,
                    "type": "text"
                }],
                "role": "user"
            }]
        }

        # Add the image entry to the dataset
        train_dataset.append(image_data)

# Check if train_dataset has enough entries before accessing the first element
if len(train_dataset) > 0:
    # Example of how the first entry in the dataset would look
    print(train_dataset[3])

    # To display the corresponding image (train_dataset[0]['images'][0])
    img_to_display = train_dataset[3]['images'][0]

    # Display the image using matplotlib
    plt.imshow(img_to_display)
    plt.axis('off')  # Hide axes for a cleaner view
    plt.show()
else:
    print("train_dataset is empty.")

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt

# Paths to your test images
image_directory = "/content/drive/MyDrive/Test_SMOL_VLM_Images"
image_files = sorted(os.listdir(image_directory))

# Define your valid image extensions
valid_extensions = (".jpg", ".jpeg")
image_files = [file for file in image_files if file.lower().endswith(valid_extensions)]

# Define your chosen responses, rejected responses, and prompts for each image
chosen_responses = [
    "Yes, there is fire visible in front of the house. The flames can be seen clearly, and it appears to be spreading across the front yard.",
    "Yes, there is fire in the front yard of the house. The grass and surrounding area appear to be burning due to intense flames.",
    "Yes, the chimney of the house has caught fire. The smoke and flames are emanating from the chimney's top, indicating an internal fire.",
    "Yes, the house appears to be on fire. Large flames are visible on the walls and roof, causing significant damage.",
    "Yes, there is fire in the car workshop. Several vehicles and equipment seem to be burning, creating heavy smoke in the area.",
    "Yes, there is fire inside the factory on an equipment. The machinery is engulfed in flames, posing a serious hazard."
]

rejected_responses = [
    "No, there is no fire visible in front of the house. The area seems clear and unaffected by any flames.",
    "No, there is no fire in the front yard of the house. The lawn and surroundings look intact and undamaged.",
    "No, the chimney of the house does not appear to be on fire. There is no smoke or flames coming from it.",
    "No, the house is not on fire. The structure appears stable and unaffected by any fire-related damage.",
    "No, there is no fire in the car workshop. The vehicles and equipment are intact, and no smoke or flames are visible.",
    "No, there is no fire inside the factory on any equipment. The machinery seems to be functioning normally without any signs of fire."
]

# Create a dataset for the first 6 images
test_dataset = []
for i, image_file in enumerate(image_files[:6]):
    img_path = os.path.join(image_directory, image_file)

    # Open the image
    image = Image.open(img_path)

    # Convert to RGB if not already
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Resize the image
    image = image.resize((500, 333))

    # Convert the image back to a JPEG format buffer to retain plugin class
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG")
    buffer.seek(0)  # Move to the beginning of the buffer

    # Reload the image from the buffer to get back the JpegImageFile type
    image = Image.open(buffer)

    # Create dataset entry
    image_data = {
        'chosen': [{
            'content': [{
                'text': chosen_responses[i],
                'type': 'text'
            }],
            'role': 'assistant'
        }],
        'rejected': [{
            'content': [{
                'text': rejected_responses[i],
                'type': 'text'
            }],
            'role': 'assistant'
        }],
        'images': [image],
        'prompt': [{
            'content': [{
                'text': None,
                'type': 'image'
            }, {
                'text': prompts[i],
                'type': 'text'
            }],
            'role': 'user'
        }]
    }

    test_dataset.append(image_data)

# Display the images with their responses
for idx, entry in enumerate(test_dataset):
    print(f"Image {idx + 1}:")
    print("Prompt:", entry['prompt'][0]['content'][1]['text'])
    print("Chosen Response:", entry['chosen'][0]['content'][0]['text'])
    print("Rejected Response:", entry['rejected'][0]['content'][0]['text'])
    print()

    # Display the image
    img_to_display = entry['images'][0]
    plt.imshow(img_to_display)
    plt.axis('off')
    plt.show()

    # Check the image type
    print("Stored Image Type:", type(img_to_display))  # Should print <PIL.JpegImagePlugin.JpegImageFile>


In [ ]:
test_dataset

In [ ]:
# Function to merge datasets
def merge_dataset(dataset):
    all_images = []
    all_chosen = []
    all_rejected = []
    all_prompts = []

    # Iterate over dataset and extract required elements
    for entry in dataset:
        all_images.extend(entry['images'])  # Flatten image list

        # Collect chosen responses
        all_chosen.extend(entry.get('chosen', []))

        # Collect rejected responses
        all_rejected.extend(entry.get('rejected', []))

        # Collect prompts
        all_prompts.extend(entry.get('prompt', []))

    # Create a structured dataset
    return {
        'images': all_images,    # Flattened list of images
        'chosen': all_chosen,    # List of chosen responses
        'rejected': all_rejected,  # List of rejected responses
        'prompts': all_prompts   # List of user prompts
    }

# Process train and test datasets
merged_train_dataset = merge_dataset(train_dataset)
merged_test_dataset = merge_dataset(test_dataset)

# Print the structured datasets
print("Merged Train Dataset:", merged_train_dataset)
print("Merged Test Dataset:", merged_test_dataset)

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

# Function to replace PIL Image objects with placeholders
def convert_images(images_list):
    return [["[PIL.Image.Image]"] for _ in images_list]  # Maintain structure

# Function to process and convert dataset to Hugging Face format
def process_dataset(merged_dataset):
    hf_dataset_dict = {
        "images": convert_images(merged_dataset["images"]),  # Convert images to placeholder
        "chosen": [entry["content"][0]["text"] if entry["content"] else None for entry in merged_dataset["chosen"]],
        "rejected": [entry["content"][0]["text"] if entry["content"] else None for entry in merged_dataset["rejected"]],
        "prompts": [entry["content"][1]["text"] if len(entry["content"]) > 1 else None for entry in merged_dataset["prompts"]],
    }

    # Convert dictionary to Hugging Face Dataset
    return Dataset.from_dict(hf_dataset_dict)

# Convert both train and test datasets
hf_train_dataset = process_dataset(merged_train_dataset)
hf_test_dataset = process_dataset(merged_test_dataset)

# Print dataset summaries
print("Train Dataset:", hf_train_dataset)
print("Test Dataset:", hf_test_dataset)

In [ ]:
# Print first item in the train dataset to check image state
print("First item from hf_train_dataset:")
print(hf_train_dataset[0])  # Accessing the first item

# Print first item in the test dataset to check image state
print("\nFirst item from hf_test_dataset:")
print(hf_test_dataset[0])  # Accessing the first item

In [ ]:
from datasets import Dataset
from PIL import Image

# Define a function to ensure image is in RGB
def ensure_rgb(example):
    # Convert the image to RGB if it's not already
    image = example['image']  # Single image in byte format
    image = Image.open(BytesIO(image))  # Convert the byte data to a PIL Image

    if image.mode != 'RGB':
        image = image.convert('RGB')  # Convert to RGB if not already

    # Convert the image back to bytes (after processing)
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG')
    example['image'] = img_byte_arr.getvalue()  # Update the image field with processed image

    return example

# Define a function to process each dataset entry
def process_data(example):
    # Example processing function
    # Add a new field that indicates the image size (for example)
    example['image_size'] = len(example['image'])  # Size of the image in bytes
    return example

# Use `.map()` function to apply the transformations on train and test datasets
hf_train_dataset = hf_train_dataset.map(ensure_rgb)  # Ensure images are in RGB format
hf_train_dataset = hf_train_dataset.map(process_data)  # Add image size to the dataset

# Check the processed data
print(hf_train_dataset)
print(hf_test_dataset)


In [ ]:
"""
from datasets import Dataset

# Convert hf_train_dataset and hf_test_dataset from lists to Hugging Face Datasets
hf_train_dataset = Dataset.from_list(hf_train_dataset)
hf_test_dataset = Dataset.from_list(hf_test_dataset)

print("Datasets converted to Hugging Face Dataset objects")
"""

In [ ]:
from PIL import Image

# Function to ensure images are in RGB format
def ensure_rgb(example):
    # Convert the placeholder to an actual PIL Image instance for demonstration
    image = example['images'][0]

    if isinstance(image, Image.Image):  # Check if it's a PIL Image
        if image.mode != 'RGB':  # Convert to RGB if not already
            image = image.convert('RGB')
        example['images'] = [image]  # Store back in the dataset

    return example

# Apply transformation using .map() on train and test datasets
hf_train_dataset = hf_train_dataset.map(ensure_rgb, num_proc=32)
hf_test_dataset = hf_test_dataset.map(ensure_rgb, num_proc=32)

# Print dataset structure after conversion
print("Train Dataset (after RGB conversion):", hf_train_dataset)
print("Test Dataset (after RGB conversion):", hf_test_dataset)

""""""

In [ ]:
from PIL import Image
import io

# Function to decode image from a local path
def decode_image_local(example):
    image_path = example['images'][0]  # Assuming the 'images' field contains the local image path
    image = Image.open(image_path).convert('RGB')  # Open the image and convert to RGB
    example['images'] = [image]  # Store the decoded image back in the 'images' field
    return example

In [ ]:
from PIL import Image

# Function to decode image from local path
def decode_image_local(example):
    try:
        # Get the image file path
        image_path = example['images'][0]  # Assuming the 'images' field contains the local image path
        image = Image.open(image_path).convert('RGB')  # Open and convert to RGB
        example['images'] = [image]  # Store the decoded image back in the 'images' field
    except Exception as e:
        print(f"Error decoding image: {e}")
    return example

In [ ]:
import requests
from PIL import Image
from io import BytesIO

# Function to decode image from URL
def decode_image_url(example):
    try:
        # Get the image URL
        image_url = example['images'][0]  # Assuming the 'images' field contains the image URL
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')  # Open and convert to RGB
        example['images'] = [image]  # Store the decoded image back in the 'images' field
    except Exception as e:
        print(f"Error downloading or decoding image: {e}")
    return example

In [ ]:
# To debug, print out examples that are causing errors
def fix_image_data_with_debug(example):
    try:
        return fix_image_data(example)
    except Exception as e:
        print(f"Error with example: {example}")
        print(f"Error message: {e}")
        return example  # Return unmodified example in case of error

hf_train_dataset = hf_train_dataset.map(fix_image_data_with_debug)

In [ ]:
# Apply image decoding for the train dataset
hf_train_dataset = hf_train_dataset.map(decode_image_local)  # Decode images from local paths

# Apply image decoding for the test dataset
hf_test_dataset = hf_test_dataset.map(decode_image_local)  # Decode images from local paths

In [ ]:
# Print dataset structure after conversion
print("Train Dataset (after RGB conversion):", hf_train_dataset)
print("Test Dataset (after RGB conversion):", hf_test_dataset)

In [ ]:
# Print first item in the train dataset to check image state
print("First item from hf_train_dataset:")
print(hf_train_dataset[0])  # Accessing the first item

# Print first item in the test dataset to check image state
print("\nFirst item from hf_test_dataset:")
print(hf_test_dataset[0])  # Accessing the first item

HERE

**Load the Quantized Model for Training**

In [ ]:
import torch
from transformers import Idefics3ForConditionalGeneration, AutoProcessor

model_id = "HuggingFaceTB/SmolVLM-Instruct"

In [ ]:
from transformers import BitsAndBytesConfig

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = Idefics3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    _attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_id)

**Set Up QLoRA and DPOConfig**

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
    use_dora=True,
    init_lora_weights="gaussian"
)

# Apply PEFT model adaptation
peft_model = get_peft_model(model, peft_config)

# Print trainable parameters
peft_model.print_trainable_parameters()

In [ ]:
from trl import DPOConfig

training_args = DPOConfig(
    output_dir="smolvlm-instruct-trl-dpo-rlaif-v",
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=4,
    dataset_num_proc=8,  # tokenization will use 8 processes
    dataloader_num_workers=8,  # data loading will use 8 workers
    logging_steps=10,
    report_to="tensorboard",
    push_to_hub=True,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=1,
    eval_steps=10,  # Steps interval for evaluation
    eval_strategy="steps",
    #remove_unused_columns=False
)

In [ ]:
# Updated function to handle NoneType in chosen and rejected
def process_dataset(merged_dataset):
    hf_dataset_dict = {
        "images": convert_images(merged_dataset["images"]),  # Convert images to URL format or paths
        "chosen": [
            entry["content"][0]["text"] if entry["content"] and entry["content"][0] else ""
            for entry in merged_dataset["chosen"]
        ],
        "rejected": [
            entry["content"][0]["text"] if entry["content"] and entry["content"][0] else ""
            for entry in merged_dataset["rejected"]
        ],
        "prompts": [
            entry["content"][1]["text"] if len(entry["content"]) > 1 else ""
            for entry in merged_dataset["prompts"]
        ],
    }

    # Convert dictionary to Hugging Face Dataset
    return Dataset.from_dict(hf_dataset_dict)

In [ ]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    peft_config=peft_config,
    tokenizer=processor,
)

In [ ]:
trainer.train()

**Save Results and Trained Model**

In [ ]:
trainer.save_model(training_args.output_dir)